<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/biplovbhandari/tensorflow-ml-models/blob/master/Illegal_Gold_Mining_WA_TF_Training.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/biplovbhandari/tensorflow-ml-models/blob/master/Illegal_Gold_Mining_WA_TF_Training.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
<td>
<a target="_blank"  href="https://nbviewer.org/github/biplovbhandari/tensorflow-ml-models/blob/master/Illegal_Gold_Mining_WA_TF_Training.ipynb?flush_cache=True"><img width=60px src="https://nbviewer.org/static/img/nav_logo.svg" /> View on nbviewer</a></td>

</table>

# Introduction

This is an Earth Engine <> TensorFlow demonstration notebook for developing a Deep Neural Networks (DNN) model for the Galamsey gold mining in West Africa. This notebook was developed as a part of the capacity building training to the West Africa Hub of the SERVIR.

Specifically, this notebook shows:

1.   Exporting training/testing data from Earth Engine in TFRecord format.
2.   Preparing the data for use in a TensorFlow model.
2.   Training and validating a simple model (Keras `Sequential` neural network and functional model) in TensorFlow.
3.   Making predictions on image data exported from Earth Engine in TFRecord format.
4.   Ingesting classified image data to Earth Engine in TFRecord format.
5.   Using AI Platform for inference.
6.   Using Vertex AI Platform for inference (coming soon).

# Setup software libraries

Import software libraries and/or authenticate as necessary.

In [ ]:
import ee
print("Using EE version ", ee.__version__)
import folium
print("Using Folium version ", folium.__version__)

from google.api_core import exceptions, retry
import google.auth
from google.colab import auth

import io
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime
from functools import partial
from pprint import pprint

import random
import requests
import tensorflow as tf
from tensorflow import keras
from keras import layers
print("Using TF version ", tf.__version__)

from typing import Dict, Iterable, List, Tuple

Using EE version  0.1.363
Using Folium version  0.14.0
Using TF version  2.12.0


## Authenticate to Colab, Cloud and Earth Engine

Identify yourself to Google Cloud, so you have access to storage and other resources.  When you run the code below, it will ask you to provide necessary permissions.  Follow the link to a page that will let you grant permission to the Cloud SDK to access your resources.

(You may need to run this again if you get a credentials error later.)

See [the EE auth reference](https://developers.google.com/earth-engine/guides/auth) for more info.

In [ ]:
# Replace your-project with your project name.
PROJECT = "servir-ee"

BUCKET = "wa-tf-training"

auth.authenticate_user()

credentials, _ = google.auth.default(
    scopes=[
        "https://www.googleapis.com/auth/cloud-platform",
        "https://www.googleapis.com/auth/earthengine",
    ]
)
ee.Initialize(
    credentials,
    project=PROJECT,
    opt_url="https://earthengine-highvolume.googleapis.com",
)

# Define variables

This set of global variables will be used throughout.  For this demo, you must have a Cloud Storage bucket into which you can write files.  ([learn more about creating Cloud Storage buckets](https://cloud.google.com/storage/docs/creating-buckets)).

In [ ]:
START = '2019-01-01'
END = '2019-12-31'

# A random spot near the study area
COORDS = [-1.981041, 6.074769]
TEST_POINT = ee.Geometry.Point(COORDS)

REGION = ee.Geometry.Polygon(
        [[[-3.2689309056402194, 8.045285352640722],
          [-3.2689309056402194, 4.693792623646198],
          [0.48839331310976064, 4.693792623646198],
          [0.48839331310976064, 8.045285352640722]]])

SCALE = 30        # Meters per pixel
VALIDATION_RATIO = 0.2
TEST_RATIO = 0.2
N_CLASSES = 2 # mining/no-mining

# Predictors.
S1_BANDS = ['VV', 'VH', 's1_ratio', 's1_ndratio']

S2_BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7',
               'B8', 'B8A', 'B9', 'B11', 'B12']

# Target variable.
OUTPUT_BANDS = ['mining']

OLD_LABEL = 'CID'

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
IMAGE_FILE_PREFIX = 'WA_image'

# The output path for the classified image (i.e. predictions) TFRecord file.
OUTPUT_IMAGE_FILE = 'gs://' + BUCKET + '/WA_Classified.TFRecord'

# Input stack.
INPUT_BANDS = S1_BANDS + S2_BANDS
BANDS = INPUT_BANDS + OUTPUT_BANDS

EPOCHS = 20
BATCH_SIZE = 32

# Inspect and get your datasets

We will use Sentinel-2 and Sentinel-1 Dataset, plus several indices for this training. Feel free to add more dataset as you see appropriate.

In [ ]:
# Helper function to set the class label (label=1)
def set_class_label(f: ee.Feature) -> ee.Feature:
    f = f.set(OUTPUT_BANDS[0], 1)
    f = f.select([OUTPUT_BANDS[0]])
    return f

In [ ]:
# Helper function to set the class label to the existing one
def set_label(f: ee.Feature) -> ee.Feature:
    f = f.set(OUTPUT_BANDS[0], f.get(OLD_LABEL))
    f = f.select([OUTPUT_BANDS[0]])
    return f

At this point, let's load our collected data of galamsey and non-galamsey points. (Check this [script](https://code.earthengine.google.com/84d3ba1026399cd581792da94a6cb36e) for the collection).

In [ ]:
user_added_galamsey = ee.FeatureCollection("projects/servir-wa/tf_training_aug_2023/user_added_galamsey")
user_added_non_galamsey = ee.FeatureCollection("projects/servir-wa/tf_training_aug_2023/user_added_non_galamsey")

In [ ]:
galamsey_pts = ee.FeatureCollection("projects/servir-wa/services/IllegalMining/2019_galamsey_pts")
pprint(galamsey_pts.aggregate_histogram(OLD_LABEL).getInfo())


{'0': 2000}


In [ ]:
galamsey_pts = galamsey_pts.map(set_class_label)

pprint(galamsey_pts.aggregate_histogram(OUTPUT_BANDS[0]).getInfo())

{'1': 2000}


In [ ]:
# old labels still exists

pprint(galamsey_pts.first().getInfo()), pprint(user_added_galamsey.first().getInfo()), pprint(user_added_non_galamsey.first().getInfo())

{'geometry': {'coordinates': [-2.0106086649987467, 5.854679287445995],
              'type': 'Point'},
 'id': '00000000000000000000',
 'properties': {'mining': 1},
 'type': 'Feature'}
{'geometry': {'coordinates': [-2.055461814492921, 6.530161515380249],
              'type': 'Point'},
 'id': '00000000000000000000',
 'properties': {'CID': 1},
 'type': 'Feature'}
{'geometry': {'coordinates': [-0.5952137390609912, 6.87835374216136],
              'type': 'Point'},
 'id': '00000000000000000000',
 'properties': {'CID': 0},
 'type': 'Feature'}


(None, None, None)

In [ ]:
user_added_galamsey = user_added_galamsey.map(set_label)
user_added_non_galamsey = user_added_non_galamsey.map(set_label)

pprint(galamsey_pts.first().getInfo()), pprint(user_added_galamsey.first().getInfo()), pprint(user_added_non_galamsey.first().getInfo())

{'geometry': {'coordinates': [-2.0106086649987467, 5.854679287445995],
              'type': 'Point'},
 'id': '00000000000000000000',
 'properties': {'mining': 1},
 'type': 'Feature'}
{'geometry': {'coordinates': [-2.055461814492921, 6.530161515380249],
              'type': 'Point'},
 'id': '00000000000000000000',
 'properties': {'mining': 1},
 'type': 'Feature'}
{'geometry': {'coordinates': [-0.5952137390609912, 6.87835374216136],
              'type': 'Point'},
 'id': '00000000000000000000',
 'properties': {'mining': 0},
 'type': 'Feature'}


(None, None, None)

## All points data

In [ ]:
all_points = galamsey_pts.merge(user_added_galamsey).merge(user_added_non_galamsey)

map = folium.Map(location=[COORDS[1], COORDS[0]], zoom_start=8)

folium.TileLayer(
    tiles=galamsey_pts.merge(user_added_galamsey).getMapId({'color': 'green'})["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="galamsey_pts",
  ).add_to(map)

folium.TileLayer(
    tiles=user_added_non_galamsey.getMapId({'color': 'red'})["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="non_galamsey_pts",
  ).add_to(map)

roi_outline = ee.Image().byte()\
    .paint(featureCollection=REGION,
           color=1,
           width=3).getMapId()

folium.TileLayer(
    tiles=roi_outline["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="REGION",
  ).add_to(map)


map.add_child(folium.LayerControl())

## Sentinel-2 surface reflectance

We will use a cloud-freee composite of Sentinel-2 surface reflectance data for predictors.  See [the Code Editor example](https://code.earthengine.google.com/?scriptPath=Examples%3ACloud%20Masking%2FSentinel2%20Cloud%20And%20Shadow) for details.

In [ ]:
def get_s2_composite(roi, start, end):
  s2c = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
  s2sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  s2c = s2c.filterBounds(roi.buffer(100, 1000)).filterDate(start, end)
  s2sr = s2sr.filterBounds(roi.buffer(100, 1000)).filterDate(start, end)

  def index_join(collection_a, collection_b, property_name):
    joined = ee.ImageCollection(
        ee.Join.saveFirst(property_name).apply(
            primary=collection_a,
            secondary=collection_b,
            condition=ee.Filter.equals(
                leftField='system:index',
                rightField='system:index')))
    return joined.map(
        lambda image: image.addBands(ee.Image(image.get(property_name))))

  def mask_image(image):
    prob = image.select('probability')
    return image.select('B.*').divide(10000).updateMask(prob.lt(50))

  with_cloud_probability = index_join(s2sr, s2c, 'cloud_probability')
  masked = ee.ImageCollection(with_cloud_probability.map(mask_image))
  return masked.select(S2_BANDS).median().float().unmask(0)

image = get_s2_composite(REGION, START, END)

In [ ]:
vis_params = {
  'min': 0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}


map = folium.Map(location=[COORDS[1], COORDS[0]], zoom_start=8)

folium.TileLayer(
    tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="image",
  ).add_to(map)

folium.TileLayer(
    tiles=roi_outline["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="REGION",
  ).add_to(map)

map.add_child(folium.LayerControl())


# Generate training data

There are a number of ways to generate training data in Earth Engine.  This notebook demonstrates using `image.sample` method to generate training data.

Divide data into training (60%), validation (20%) and testing (20%) datasets. Also get the distribution of the galamsey and non-galamsey points in each of the dataset.

In [ ]:
all_points = all_points.randomColumn("random", 100)

pprint(all_points.first().getInfo())

pprint(("Sample size:", all_points.size().getInfo()))

training_sample_locations = all_points.filter(ee.Filter.gt("random", VALIDATION_RATIO + TEST_RATIO)) # > 0.4
training_sample_size = training_sample_locations.size().getInfo()
pprint(("Training sample size:", training_sample_size))

validation_sample_locations = all_points.filter(ee.Filter.lte("random", VALIDATION_RATIO)) # <= 0.2
validation_sample_size = validation_sample_locations.size().getInfo()
pprint(("Validation sample size:", validation_sample_size))

test_sample_locations = all_points.filter(ee.Filter.And(ee.Filter.gt("random", VALIDATION_RATIO),
                                                        ee.Filter.lte("random", VALIDATION_RATIO + TEST_RATIO))) # > 0.2 and <= 0.4
test_sample_size = validation_sample_locations.size().getInfo()
pprint(("Test sample size:", test_sample_size))

pprint(("training_sample_locations", training_sample_locations.aggregate_histogram(OUTPUT_BANDS[0]).getInfo()))
pprint(("validation_sample_locations", validation_sample_locations.aggregate_histogram(OUTPUT_BANDS[0]).getInfo()))
pprint(("test_sample_locations", test_sample_locations.aggregate_histogram(OUTPUT_BANDS[0]).getInfo()))


{'geometry': {'coordinates': [-2.0106086649987467, 5.854679287445995],
              'type': 'Point'},
 'id': '1_1_00000000000000000000',
 'properties': {'mining': 1, 'random': 0.4438691942788049},
 'type': 'Feature'}
('Sample size:', 3132)
('Training sample size:', 1901)
('Validation sample size:', 589)
('Test sample size:', 589)
('training_sample_locations', {'0': 629, '1': 1272})
('validation_sample_locations', {'0': 189, '1': 400})
('test_sample_locations', {'0': 194, '1': 448})


In [ ]:
map = folium.Map(location=[COORDS[1], COORDS[0]], zoom_start=8)

folium.TileLayer(
    tiles=training_sample_locations.getMapId({'color': 'green'})["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="training_sample_locations",
  ).add_to(map)

folium.TileLayer(
    tiles=validation_sample_locations.getMapId({'color': 'red'})["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="validation_sample_locations",
  ).add_to(map)

folium.TileLayer(
    tiles=test_sample_locations.getMapId({'color': 'blue'})["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="test_sample_locations",
  ).add_to(map)

folium.TileLayer(
    tiles=roi_outline["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="REGION",
  ).add_to(map)

map.add_child(folium.LayerControl())


Paint the label data into an image and add that as a band to the S2 Image

In [ ]:
label_img = ee.Image().byte().paint(featureCollection=all_points, color=OUTPUT_BANDS[0]).rename(OUTPUT_BANDS[0])
image = image.addBands(label_img)

## Export Image before sampling

If you try to sample without exporting you will run into Computational Time Out. This is because there are a lot of processing going on with the S2 data using cloud score etc. So better to export the image to avoid the Computational Time out issue.

In [ ]:
def export_image_to_asset(image: ee.Image, start_training: bool, **kwargs: dict) -> ee.batch.Task:
    asset_id = kwargs.get("asset_id", "")
    print(f"Exporting image to {asset_id}..")

    training_task = ee.batch.Export.image.toAsset(
        image=image,
        description=kwargs.get("description", "myExportImageTask"),
        assetId=asset_id,
        region=kwargs.get("region", None),
        scale=kwargs.get("scale", SCALE),
        maxPixels=kwargs.get("max_pixels", 1E13),
    )
    if start_training: training_task.start()
    return training_task

In [ ]:
# Change False to True if you're actually exporting
export_image = False
training_task = export_image_to_asset(image, export_image, **{"asset_id": "projects/servir-wa/tf_training_aug_2023/s2_2019_demo", "region": REGION})


Exporting image to projects/servir-wa/tf_training_aug_2023/s2_2019_demo..


In [ ]:
#@title Don't run unless you are exporting the image

# Print all tasks.
# print(ee.batch.Task.list())

# Poll the training task until it's done.
import time
while training_task.active():
  print('Polling for task (id: {}).'.format(training_task.id))
  time.sleep(30)
print('Done with training export.')

Finally assemble all the data for the sampling purpose. Here we are adding already exported Sentinel-2 and Sentinel-1 dataset.

In [ ]:
s2 = ee.Image("projects/servir-wa/tf_training_aug_2023/s2_2019_demo")
# s1 = ee.Image("projects/servir-wa/tf_training_aug_2023/s1_asc_2019_demo")
s1 = ee.Image("projects/servir-wa/tf_training_aug_2023/s1_asc_2019_demo_30_m")
s1 = s1.select(['VV', 'VH', 'ratio', 'ndratio'], S1_BANDS)
image = s1.addBands(s2)
pprint(image.bandNames().getInfo())

['VV',
 'VH',
 's1_ratio',
 's1_ndratio',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'mining']


In [ ]:
def sample_image(image: ee.Image, region: ee.FeatureCollection, **kwargs: dict) -> ee.FeatureCollection:
    sample = image.sample(region=ee.FeatureCollection(region),
                          scale=kwargs.get("seed", SCALE),
                          seed=kwargs.get("seed", 100),
                          geometries=kwargs.get("geometries", False))
    return sample

In [ ]:
training_samples = sample_image(image, training_sample_locations)
validation_samples = sample_image(image, validation_sample_locations)
test_samples = sample_image(image, test_sample_locations)


In [ ]:
# sanity check before exporting them
example_samples = sample_image(image, ee.FeatureCollection(ee.Feature(training_sample_locations.first())), **{"scale": SCALE})
pprint(example_samples.first().getInfo())

{'geometry': None,
 'id': '0',
 'properties': {'B1': 0.07715000212192535,
                'B11': 0.2533000111579895,
                'B12': 0.16979999840259552,
                'B2': 0.08049999922513962,
                'B3': 0.11225000023841858,
                'B4': 0.12084999680519104,
                'B5': 0.1732500046491623,
                'B6': 0.243599995970726,
                'B7': 0.26205000281333923,
                'B8': 0.25804999470710754,
                'B8A': 0.28334999084472656,
                'B9': 0.30869999527931213,
                'VH': -14.61683173873444,
                'VV': -8.161584434856472,
                'mining': 1,
                's1_ndratio': -0.28339315844804536,
                's1_ratio': 0.5583689120008384},
 'type': 'Feature'}


In [ ]:
def export_collection_to_cloud_storage(collection: ee.FeatureCollection, start_training: bool, **kwargs: dict) -> ee.batch.Task:
    description = kwargs.get("description", "myExportTableTask")
    bucket = kwargs.get("bucket", "myBucket")
    file_prefix = kwargs.get("file_prefix") if kwargs.get("file_prefix") is not None else description
    print(f"Exporting training data to gs://{bucket}/{file_prefix}..")
    training_task = ee.batch.Export.table.toCloudStorage(
        collection=collection,
        description=description,
        fileNamePrefix=file_prefix,
        bucket=bucket,
        fileFormat=kwargs.get("file_format", "TFRecord"),
        selectors=kwargs.get("selectors", collection.first().propertyNames().getInfo()),
    )
    if start_training: training_task.start()
    return training_task

In [ ]:
# Names for output files.
train_file_prefix = "illegal_mining_training"
test_file_prefix = "illegal_mining_testing"
validate_file_prefix = "illegal_mining_validation"


Finally, let's export the samples in TFRecord format for using with the DL.

In [ ]:
# Change start_training=True if you're actually exporting
start_training = False

kwargs = { "bucket": BUCKET, "selectors": BANDS }
training_task = export_collection_to_cloud_storage(training_samples, start_training=start_training, **{**kwargs, "file_prefix": train_file_prefix, "description": "Training"})
testing_task = export_collection_to_cloud_storage(test_samples, start_training=start_training, **{**kwargs, "file_prefix": test_file_prefix, "description": "Testing"})
validation_task = export_collection_to_cloud_storage(validation_samples, start_training=start_training, **{**kwargs, "file_prefix": validate_file_prefix, "description": "Validation"})


Exporting training data to gs://wa-tf-training/illegal_mining_training..
Exporting training data to gs://wa-tf-training/illegal_mining_testing..
Exporting training data to gs://wa-tf-training/illegal_mining_validation..


In [ ]:
#@title Don't run unless you are exporting the training data

# print(ee.batch.Task.list())

# Poll the training task until it's done.
import time
while training_task.active():
  print('Polling for task (id: {}).'.format(training_task.id))
  time.sleep(30)
print('Done with training export.')

while testing_task.active():
  print('Polling for task (id: {}).'.format(testing_task.id))
  time.sleep(30)
print('Done with testing export.')


while validation_task.active():
  print('Polling for task (id: {}).'.format(validation_task.id))
  time.sleep(30)
print('Done with validation export.')



### Check existence of the exported files

If you've seen the status of the export tasks change to `COMPLETED`, then check for the existince of the files in the output Cloud Storage bucket.

In [ ]:
file_name_suffix = '.tfrecord.gz'
train_file_path = 'gs://' + BUCKET + '/' + train_file_prefix + file_name_suffix
test_file_path = 'gs://' + BUCKET + '/' + test_file_prefix + file_name_suffix
validate_file_path = 'gs://' + BUCKET + '/' + validate_file_prefix + file_name_suffix

print('Found training file.' if tf.io.gfile.exists(train_file_path) else 'No training file found.')
print('Found testing file.' if tf.io.gfile.exists(test_file_path) else 'No testing file found.')
print('Found validation file.' if tf.io.gfile.exists(validate_file_path) else 'No validation file found.')

Found training file.
Found testing file.
Found validation file.


## Define the structure of your data

For parsing the exported TFRecord files, `FEATURES_DICT` is a mapping between feature names (recall that `BANDS` contains the band and label names) and `float32` [`tf.io.FixedLenFeature`](https://www.tensorflow.org/api_docs/python/tf/io/FixedLenFeature) objects.  This mapping is necessary for telling TensorFlow how to read data in a TFRecord file into tensors.  Specifically, **all numeric data exported from Earth Engine is exported as `float32`**.

(Note: *features* in the TensorFlow context (i.e. [`tf.train.Feature`](https://www.tensorflow.org/api_docs/python/tf/train/Feature)) are not to be confused with Earth Engine features (i.e. [`ee.Feature`](https://developers.google.com/earth-engine/api_docs#eefeature)), where the former is a protocol message type for serialized data input to the model and the latter is a geometry-based geographic data structure.)

In [ ]:
COLUMNS = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
  for k in BANDS
]

FEATURES_DICT = dict(zip(BANDS, COLUMNS))

# Data preparation and pre-processing

Read data from the TFRecord file into a `tf.data.Dataset`.  Pre-process the dataset to get it into a suitable format for input to the model.

In [ ]:
def create_tfrecord_from_file(filename: str) -> tf.data.TFRecordDataset:
    return tf.data.TFRecordDataset(filename, compression_type="GZIP")

## Read into a `tf.data.Dataset`

Here we are going to read a file in Cloud Storage into a `tf.data.Dataset`.  ([these TensorFlow docs](https://www.tensorflow.org/guide/data) explain more about reading data into a `Dataset`).  Check that you can read examples from the file.  The purpose here is to ensure that we can read from the file without an error.  The actual content is not necessarily human readable.

In [ ]:
train_dataset = tf.data.Dataset.list_files(train_file_path).interleave(create_tfrecord_from_file)
test_dataset = tf.data.Dataset.list_files(test_file_path).interleave(create_tfrecord_from_file)
validate_dataset = tf.data.Dataset.list_files(validate_file_path).interleave(create_tfrecord_from_file)

## Parse the dataset

Now we need to make a parsing function for the data in the TFRecord files.  The data comes in flattened 2D arrays per record and we want to use the first part of the array for input to the model and the last element of the array as the class label.  The parsing function reads data from a serialized [`Example` proto](https://www.tensorflow.org/api_docs/python/tf/train/Example) into a dictionary in which the keys are the feature names and the values are the tensors storing the value of the features for that example.  ([These TensorFlow docs](https://www.tensorflow.org/tutorials/load_data/tfrecord) explain more about reading `Example` protos from TFRecord files).

In [ ]:
def create_tfrecord_from_file(filename: str) -> tf.data.TFRecordDataset:
    return tf.data.TFRecordDataset(filename, compression_type="GZIP")

In [ ]:
def parse_tfrecord_dnn(example_proto: tf.Tensor) -> Tuple:
    parsed_features = tf.io.parse_single_example(example_proto, FEATURES_DICT)
    label = parsed_features.pop(OUTPUT_BANDS[0])
    label = tf.cast(label, tf.int32)
    return parsed_features, label

In [ ]:
def to_tuple_dnn(dataset: dict, label: tf.Tensor, depth: int = 1) -> Tuple:
    return tf.transpose(list(dataset.values())), tf.one_hot(indices=label, depth=depth)

In [ ]:
parser = partial(parse_tfrecord_dnn)
tupler = partial(to_tuple_dnn, depth=N_CLASSES)

In [ ]:
train_dataset = train_dataset.map(parser, num_parallel_calls=tf.data.AUTOTUNE)


In [ ]:
pprint(iter(train_dataset).next())

({'B1': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.04895], dtype=float32)>,
  'B11': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.02475], dtype=float32)>,
  'B12': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0203], dtype=float32)>,
  'B2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0496], dtype=float32)>,
  'B3': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.05795], dtype=float32)>,
  'B4': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.04185], dtype=float32)>,
  'B5': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.04425], dtype=float32)>,
  'B6': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0384], dtype=float32)>,
  'B7': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.03915], dtype=float32)>,
  'B8': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.03395], dtype=float32)>,
  'B8A': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.03505], dtype=float32)>,
  'B9': <tf.Tensor: shape=(1,), dtype=float

Note that each record of the parsed dataset contains a tuple.  The first element of the tuple is a dictionary with bands for keys and the numeric value of the bands for values.  The second element of the tuple is a class label.

## Create additional features

Another thing we might want to do as part of the input process is to create new features, for example NDVI, a vegetation index computed from reflectance in two spectral bands.  Here are some helper functions for that.

In [ ]:
def normalized_difference(a, b):
    nd = (a - b) / (a + b)
    nd_inf = (a - b) / (a + b + 0.000001)
    return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def add_NDVI(features, label):
    NIR = features["B5"]
    RED = features["B4"]
    features["NDVI"] = normalized_difference(NIR, RED)
    return features, label

def add_EVI(features, label):
    NIR = features["B5"]
    RED = features["B4"]
    BLUE = features["B2"]
    EVI = 2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))
    features["EVI"] = EVI
    return features, label

def add_MNDWI(features, label):
    GREEN = features["B3"]
    NIR = features["B5"]
    MNDWI = normalized_difference(GREEN, NIR)
    features["MNDWI"] = MNDWI
    return features, label

def add_SAVI(features, label):
    NIR = features["B5"]
    RED = features["B4"]
    SAVI = (NIR - RED) * (1 + 0.5)/(NIR + RED + 0.5)
    features["SAVI"] = SAVI
    return features, label

def add_IBI(features, label):
    #Add Index-Based Built-Up Index (IBI)
    RED = features["B4"]
    GREEN = features["B3"]
    SWIR1 = features["B11"]
    NIR = features["B5"]

    ibiA = (2 * SWIR1) / (SWIR1 + NIR)
    ibiB = (NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1))
    IBI = normalized_difference(ibiA, ibiB)
    features["IBI"] = IBI
    return features, label


In [ ]:
train_dataset = train_dataset.map(add_NDVI)
train_dataset = train_dataset.map(add_EVI)
train_dataset = train_dataset.map(add_MNDWI)
train_dataset = train_dataset.map(add_SAVI)
train_dataset = train_dataset.map(add_IBI)


train_dataset = train_dataset.map(tupler, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(512)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
train_dataset = train_dataset.repeat()

In [ ]:
validate_dataset = validate_dataset.map(parser, num_parallel_calls=tf.data.AUTOTUNE)
validate_dataset = validate_dataset.map(add_NDVI)
validate_dataset = validate_dataset.map(add_EVI)
validate_dataset = validate_dataset.map(add_MNDWI)
validate_dataset = validate_dataset.map(add_SAVI)
validate_dataset = validate_dataset.map(add_IBI)

validate_dataset = validate_dataset.map(tupler, num_parallel_calls=tf.data.AUTOTUNE)
validate_dataset = validate_dataset.shuffle(512)
validate_dataset = validate_dataset.batch(1)
validate_dataset = validate_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
validate_dataset = validate_dataset.repeat()

# Model setup

The basic workflow for classification in TensorFlow is:

1.  Create the model.
2.  Train the model (i.e. `fit()`).
3.  Use the trained model for inference (i.e. `predict()`).

Here we'll create a `Sequential` neural network model using Keras.  This simple model is inspired by examples in:

* [The TensorFlow Get Started tutorial](https://www.tensorflow.org/tutorials/)
* [The TensorFlow Keras guide](https://www.tensorflow.org/guide/keras#build_a_simple_model)
* [The Keras `Sequential` model examples](https://keras.io/getting-started/sequential-model-guide/#multilayer-perceptron-mlp-for-multi-class-softmax-classification)

Note that the model used here is purely for demonstration purposes and hasn't gone through any performance tuning.

We will look at both Sequential and Functional based approach to making models.

## 1.   Sequential Model with keras

In [ ]:
# Define the layers in the model.
model1 = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation="relu"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(32, activation="relu"),
  tf.keras.layers.Dropout(0.2),
  # 2 class = sigmoid, multi-class = softmax
  tf.keras.layers.Dense(N_CLASSES, activation="sigmoid")
])

# Compile the model with the specified loss function.
model1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [ ]:
# Fit the model to the training data.
history1 = model1.fit(
            x=train_dataset,
            epochs=EPOCHS,
            steps_per_epoch=(training_sample_size // BATCH_SIZE),
            validation_data=validate_dataset,
            validation_steps=(validation_sample_size // BATCH_SIZE)
        )

Epoch 1/25
59/59 [==============================] - 11s 24ms/step - loss: 0.6726 - accuracy: 0.6483 - val_loss: 0.5854 - val_accuracy: 0.7778
Epoch 2/25
59/59 [==============================] - 1s 22ms/step - loss: 0.6355 - accuracy: 0.6752 - val_loss: 0.5319 - val_accuracy: 0.7778
Epoch 3/25
59/59 [==============================] - 1s 22ms/step - loss: 0.5979 - accuracy: 0.7250 - val_loss: 0.5737 - val_accuracy: 0.7778
Epoch 4/25
59/59 [==============================] - 2s 33ms/step - loss: 0.5410 - accuracy: 0.7533 - val_loss: 0.4579 - val_accuracy: 0.7778
Epoch 5/25
59/59 [==============================] - 2s 27ms/step - loss: 0.4713 - accuracy: 0.7988 - val_loss: 0.3266 - val_accuracy: 0.8333
Epoch 6/25
59/59 [==============================] - 2s 33ms/step - loss: 0.4107 - accuracy: 0.8283 - val_loss: 0.1976 - val_accuracy: 0.9444
Epoch 7/25
59/59 [==============================] - 1s 19ms/step - loss: 0.3779 - accuracy: 0.8459 - val_loss: 0.1966 - val_accuracy: 0.9444
Epoch 8/25
5

In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 128)            2816      
                                                                 
 dropout (Dropout)           (None, 1, 128)            0         
                                                                 
 dense_1 (Dense)             (None, 1, 64)             8256      
                                                                 
 dropout_1 (Dropout)         (None, 1, 64)             0         
                                                                 
 dense_2 (Dense)             (None, 1, 32)             2080      
                                                                 
 dropout_2 (Dropout)         (None, 1, 32)             0         
                                                                 
 dense_3 (Dense)             (None, 1, 2)              6

## 2. Functional Model with keras

In [ ]:
inputs = keras.Input(shape=(None, len(INPUT_BANDS) + 5), name="input_layer")
y = keras.layers.Conv1D(128, 3, activation="relu", padding="same", name="conv1")(inputs)
y = keras.layers.MaxPooling1D(2, padding="same")(y)
y = keras.layers.Conv1D(64, 3, activation="relu", padding="same", name="conv2")(y)
y = keras.layers.MaxPooling1D(2, padding="same")(y)
y = keras.layers.Conv1D(len(INPUT_BANDS) + 5, 2, activation="relu", padding="same", name="conv4")(y)

all_inputs = keras.layers.concatenate([inputs, y])

x = keras.layers.Dense(128, activation="relu")(all_inputs)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)
output = keras.layers.Dense(N_CLASSES, activation="sigmoid")(x)

model2 = keras.models.Model(inputs=inputs, outputs=output)

# Compile the model with the specified loss function.
model2.compile(optimizer=tf.keras.optimizers.Adam(),
               loss='binary_crossentropy',
               metrics=['accuracy'])


In [ ]:
model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, None, 21)]   0           []                               
                                                                                                  
 conv1 (Conv1D)                 (None, None, 128)    8192        ['input_layer[0][0]']            
                                                                                                  
 max_pooling1d_4 (MaxPooling1D)  (None, None, 128)   0           ['conv1[0][0]']                  
                                                                                                  
 conv2 (Conv1D)                 (None, None, 64)     24640       ['max_pooling1d_4[0][0]']        
                                                                                            

In [ ]:
# Fit the model to the training data.
history2 = model2.fit(
            x=train_dataset,
            epochs=EPOCHS,
            steps_per_epoch=(training_sample_size // BATCH_SIZE),
            validation_data=validate_dataset,
            validation_steps=(validation_sample_size // BATCH_SIZE)
        )

Epoch 1/25
59/59 [==============================] - 13s 31ms/step - loss: 0.7147 - accuracy: 0.6298 - val_loss: 0.6391 - val_accuracy: 0.6667
Epoch 2/25
59/59 [==============================] - 2s 24ms/step - loss: 0.6520 - accuracy: 0.6768 - val_loss: 0.6339 - val_accuracy: 0.6667
Epoch 3/25
59/59 [==============================] - 1s 22ms/step - loss: 0.6116 - accuracy: 0.7159 - val_loss: 0.6631 - val_accuracy: 0.7778
Epoch 4/25
59/59 [==============================] - 3s 43ms/step - loss: 0.5224 - accuracy: 0.7822 - val_loss: 0.5511 - val_accuracy: 0.7222
Epoch 5/25
59/59 [==============================] - 1s 23ms/step - loss: 0.4045 - accuracy: 0.8432 - val_loss: 0.8022 - val_accuracy: 0.6111
Epoch 6/25
59/59 [==============================] - 1s 23ms/step - loss: 0.3785 - accuracy: 0.8502 - val_loss: 0.1110 - val_accuracy: 1.0000
Epoch 7/25
59/59 [==============================] - 1s 21ms/step - loss: 0.3380 - accuracy: 0.8609 - val_loss: 0.0725 - val_accuracy: 1.0000
Epoch 8/25
5

## Check model accuracy on the test set

Now that we have a trained model, we can evaluate it using the test dataset.  To do that, read and prepare the test dataset in the same way as the training dataset.  Here we specify a batch size of 1 so that each example in the test set is used exactly once to compute model accuracy.

In [ ]:
test_dataset = test_dataset.map(parser, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(add_NDVI)
test_dataset = test_dataset.map(add_EVI)
test_dataset = test_dataset.map(add_MNDWI)
test_dataset = test_dataset.map(add_SAVI)
test_dataset = test_dataset.map(add_IBI)

test_dataset = test_dataset.map(tupler, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(1)

In [ ]:
model1.evaluate(test_dataset)

642/642 [==============================] - 7s 10ms/step - loss: 0.2569 - accuracy: 0.8863


[0.25685566663742065, 0.8862928152084351]

In [ ]:
model2.evaluate(test_dataset)

642/642 [==============================] - 4s 6ms/step - loss: 0.2922 - accuracy: 0.8551


[0.29222628474235535, 0.855140209197998]

# Inferences

## 1.   Use the trained model to classify an image from Earth Engine



Now it's time to classify the image that was exported from Earth Engine.  If the exported image is large, it will be split into multiple TFRecord files in its destination folder.  There will also be a JSON sidecar file called "the mixer" that describes the format and georeferencing of the image.  Here we will find the image files and the mixer file, getting some info out of the mixer that will be useful during model inference.

### Export the imagery

You also need to export imagery using TFRecord format.  Specifically, export whatever imagery you want to be classified by the trained model into the output Cloud Storage bucket.

In [ ]:
IMAGE_FILE_PREFIX

'WA_image'

In [ ]:
def export_image_to_cloud_storage(image: ee.Image, start_training: bool, region: ee.Geometry, **kwargs: dict) -> ee.batch.Task:

    print(f"Exporting image to BUCKET {BUCKET}..")

    # Specify patch and file dimensions.
    image_export_options = {
    "patchDimensions": [256, 256],
    "maxFileSize": 104857600,
    "compressed": True
    }

    training_task = ee.batch.Export.image.toCloudStorage(
        image=image,
        description=kwargs.get("description", "myExportImageTask"),
        fileNamePrefix=kwargs.get("file_name_prefix", IMAGE_FILE_PREFIX),
        bucket=BUCKET,
        fileFormat="TFRecord",
        region=region.getInfo()['coordinates'],
        scale=kwargs.get("scale", SCALE),
        maxPixels=kwargs.get("max_pixels", 1E13),
        formatOptions=image_export_options,
    )
    if start_training: training_task.start()
    return training_task

In [ ]:
# Getting a smaller area for prediction because this might take long to complete

EXPORT_REGION = ee.Geometry.Polygon(
        [[[-2.2499489720464894, 6.806096384151279],
          [-2.2499489720464894, 5.675769064032216],
          [-1.3435769017339894, 5.675769064032216],
          [-1.3435769017339894, 6.806096384151279]]])



In [ ]:
start_cloud_export = False
export_task = export_image_to_cloud_storage(image, start_training=start_cloud_export, region=EXPORT_REGION)

Exporting image to BUCKET wa-tf-training..


In [ ]:
#@title Don't run unless you are exporting the image

# Print all tasks.
# print(ee.batch.Task.list())

# Poll the training task until it's done.
import time
while export_task.active():
  print('Polling for task (id: {}).'.format(training_task.id))
  time.sleep(30)
print('Done with training export.')

### Find the image files and JSON mixer file in Cloud Storage

Use `gsutil` to locate the files of interest in the output Cloud Storage bucket.  Check to make sure your image export task finished before running the following.


In [ ]:
# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'gs://'{BUCKET}
# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if IMAGE_FILE_PREFIX in s]

# Get the list of image files and the JSON mixer file.
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

# Make sure the files are in the right order.
image_files_list.sort()

pprint(image_files_list)
pprint(json_file)

['gs://wa-tf-training/WA_image00000.tfrecord.gz',
 'gs://wa-tf-training/WA_image00001.tfrecord.gz',
 'gs://wa-tf-training/WA_image00002.tfrecord.gz',
 'gs://wa-tf-training/WA_image00003.tfrecord.gz',
 'gs://wa-tf-training/WA_image00004.tfrecord.gz',
 'gs://wa-tf-training/WA_image00005.tfrecord.gz',
 'gs://wa-tf-training/WA_image00006.tfrecord.gz',
 'gs://wa-tf-training/WA_image00007.tfrecord.gz',
 'gs://wa-tf-training/WA_image00008.tfrecord.gz']
'gs://wa-tf-training/WA_imagemixer.json'


### Read the JSON mixer file

The mixer contains metadata and georeferencing information for the exported patches, each of which is in a different file.  Read the mixer to get some information needed for prediction.

In [ ]:
import json

# Load the contents of the mixer file to a JSON object.
json_text = !gsutil cat {json_file}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(json_text.nlstr)
pprint(mixer)

{'patchDimensions': [256, 256],
 'patchesPerRow': 13,
 'projection': {'affine': {'doubleMatrix': [0.0002694945852358564,
                                            0.0,
                                            -2.2500102921341663,
                                            0.0,
                                            -0.0002694945852358564,
                                            6.806355244716791]},
                'crs': 'EPSG:4326'},
 'totalPatches': 208}


### Read the image files into a dataset

You can feed the list of files directly to the `TFRecordDataset` constructor to make a combined dataset on which to perform inference.  The input needs to be preprocessed differently than the training and testing.  Mainly, this is because the pixels are written into records as patches, we need to read the patches in as one big tensor (one patch for each band), then flatten them into lots of little tensors.

In [ ]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32)
    for k in BANDS
]

# Parsing dictionary.
image_features_dict = dict(zip(INPUT_BANDS, image_columns))

# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Add additional features (NDVI).
image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_NDVI(features, None)[0]
)

image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_EVI(features, None)[0]
)

image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_MNDWI(features, None)[0]
)

image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_SAVI(features, None)[0]
)

image_dataset = image_dataset.map(
  # Add NDVI to a feature that doesn't have a label.
  lambda features: add_IBI(features, None)[0]
)


# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)

### Generate predictions for the image pixels

To get predictions in each pixel, run the image dataset through the trained model using `model.predict()`.  Print the first prediction to see that the output is a list of the three class probabilities for each pixel.  Running all predictions might take a while.

In [ ]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model1.predict(image_dataset, steps=patches, verbose=1)

# Note that the predictions come as a numpy array.  Check the first one.
print(predictions[0])

208/208 [==============================] - 2889s 14s/step
[[0.8816099 0.1042067]]


In [ ]:
pprint(len(predictions)), pprint(predictions[3045])

13631488
array([[0.67055327, 0.31307673]], dtype=float32)


(None, None)

### Write the predictions to a TFRecord file

Now that there's a list of class probabilities in `predictions`, it's time to write them back into a file, optionally including a class label which is simply the index of the maximum probability.  We'll write directly from TensorFlow to a file in the output Cloud Storage bucket.

Iterate over the list, compute class label and write the class and the probabilities in patches.  Specifically, we need to write the pixels into the file as patches in the same order they came out.  The records are written as serialized `tf.train.Example` protos.  This might take a while.

In [ ]:
# Instantiate the writer.
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[], [], []]
cur_patch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1).numpy()[0])
  patch[1].append(prediction[0][0])
  patch[2].append(prediction[0][1])
  # Once we've seen a patches-worth of class_ids...
  if (len(patch[0]) == patch_width * patch_height):
    print(f"Done with patch {cur_patch} of {patches} ...")
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          "prediction": tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          "nonMiningProb": tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          "miningProb": tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2]))
        }
      )
    )
    # Write the example to the file and clear our patch array so it's ready for
    # another batch of class ids
    writer.write(example.SerializeToString())
    patch = [[], [], []]
    cur_patch += 1

writer.close()

Done with patch 1 of 208 ...
Done with patch 2 of 208 ...
Done with patch 3 of 208 ...
Done with patch 4 of 208 ...
Done with patch 5 of 208 ...
Done with patch 6 of 208 ...
Done with patch 7 of 208 ...
Done with patch 8 of 208 ...
Done with patch 9 of 208 ...
Done with patch 10 of 208 ...
Done with patch 11 of 208 ...
Done with patch 12 of 208 ...
Done with patch 13 of 208 ...
Done with patch 14 of 208 ...
Done with patch 15 of 208 ...
Done with patch 16 of 208 ...
Done with patch 17 of 208 ...
Done with patch 18 of 208 ...
Done with patch 19 of 208 ...
Done with patch 20 of 208 ...
Done with patch 21 of 208 ...
Done with patch 22 of 208 ...
Done with patch 23 of 208 ...
Done with patch 24 of 208 ...
Done with patch 25 of 208 ...
Done with patch 26 of 208 ...
Done with patch 27 of 208 ...
Done with patch 28 of 208 ...
Done with patch 29 of 208 ...
Done with patch 30 of 208 ...
Done with patch 31 of 208 ...
Done with patch 32 of 208 ...
Done with patch 33 of 208 ...
Done with patch 34 

### Upload the classifications to an Earth Engine asset

#### Verify the existence of the predictions file

At this stage, there should be a predictions TFRecord file sitting in the output Cloud Storage bucket.  Use the `gsutil` command to verify that the predictions image (and associated mixer JSON) exist and have non-zero size.

In [ ]:
!gsutil ls -l {OUTPUT_IMAGE_FILE}

 122705648  2023-08-15T19:25:16Z  gs://wa-tf-training/WA_Classified.TFRecord
TOTAL: 1 objects, 122705648 bytes (117.02 MiB)


### Upload the classified image to Earth Engine

Upload the image to Earth Engine directly from the Cloud Storage bucket with the [`earthengine` command](https://developers.google.com/earth-engine/command_line#upload).  Provide both the image TFRecord file and the JSON file as arguments to `earthengine upload`.

In [ ]:
OUTPUT_ASSET_ID = "projects/servir-wa/tf_training_aug_2023/tf_training_prediction_example"
print('Uploading to ' + OUTPUT_ASSET_ID)

Uploading to projects/servir-wa/tf_training_aug_2023/tf_training_prediction_example


In [ ]:
# Start the upload.
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

Started upload task with ID: 6O5HX5D6HJWA4OC3VIOO7H3J


If the upload doesn't work, you would need to authenticate again with earthengine using `ee.Authenticate()`

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=H-Vz0BkJ29dwEiqDHm7rJ40rvXBLTrPv4pNzIB80j_w&tc=Lr6Oaf-fstI2udOGnjFJSA2tCk3ykToxVn4I5fV6V48&cc=te9e6iHe1OjeuO8QqdXK24IYLDFkcJD1OoHlcJqq46o

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BU2RIBeAlurwEBu5hY89Se05ozGmSWeElJZ1noxnuAcnbkGRUzQAtM

Successfully saved authorization token.


### Check the status of the asset ingestion

You can also use the Earth Engine API to check the status of your asset upload.  It might take a while.  The upload of the image is an asset ingestion task.

In [ ]:
ee.batch.Task.list()

### View the ingested asset

Display the vector of class probabilities as an RGB image with colors corresponding to the probability of bare, vegetation, water in a pixel.  Also display the winning class using the same color palette.

In [ ]:
predictions_image = ee.Image(OUTPUT_ASSET_ID)

prediction_vis = {
  "bands": "prediction",
  "min": 0,
  "max": 1,
}
mining_probability_vis = {"bands": ["miningProb"], "min": 0, "max": 1}
non_mining_probability_vis = {"bands": ["nonMiningProb"], "min": 0, "max": 1}

prediction_map_id = predictions_image.getMapId(prediction_vis)
mining_probability_map_id = predictions_image.getMapId(mining_probability_vis)
non_mining_probability_map_id = predictions_image.getMapId(non_mining_probability_vis)

map = folium.Map(location=[COORDS[1], COORDS[0]])

folium.TileLayer(
  tiles=prediction_map_id["tile_fetcher"].url_format,
  attr="Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
  overlay=True,
  name="Prediction",
).add_to(map)

folium.TileLayer(
  tiles=mining_probability_map_id["tile_fetcher"].url_format,
  attr="Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
  overlay=True,
  name="Mining Probability",
).add_to(map)

folium.TileLayer(
  tiles=non_mining_probability_map_id["tile_fetcher"].url_format,
  attr="Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
  overlay=True,
  name="Non-Mining Probability",
).add_to(map)

map.add_child(folium.LayerControl())
map

The result does not look terrible. But this can always be improved with more training data or adding new features.
You can see the difference in using only Sentinel-2 vs adding Sentinel-1 in this [script](https://code.earthengine.google.com/bbf335ba829569d6ac8f0f3c5cedc063).

## 2. Using the AI Platform

Note that the notebook VM is sometimes not heavy-duty enough to get through a whole training job, especially if you have a large buffer size or a large number of epochs. You can still use this notebook for training, but may need to set up an alternative VM (learn more) for production use. Alternatively, you can package your code for running large training jobs on Google's AI Platform as described here. The following code loads a pre-trained model, which you can use for predictions right away.

In [ ]:
MODEL_DIR = "gs://" + BUCKET + "/model"
print(f"Saving model to {MODEL_DIR}")
model1.save(MODEL_DIR)

Saving model to gs://wa-tf-training/model


### Prepare the model for making predictions in Earth Engine

Before we can use the model in Earth Engine, it needs to be hosted by AI Platform.  But before we can host the model on AI Platform we need to *EEify* (a new word!) it.  The EEification process merely appends some extra operations to the input and outputs of the model in order to accomdate the interchange format between pixels from Earth Engine (float32) and inputs to AI Platform (base64).  (See [this doc](https://cloud.google.com/ml-engine/docs/online-predict#binary_data_in_prediction_input) for details.)  

## `earthengine model prepare`
The EEification process is handled for you using the Earth Engine command `earthengine model prepare`.  To use that command, we need to specify the input and output model directories and the name of the input and output nodes in the TensorFlow computation graph.  We can do all that programmatically:

In [ ]:
# ee.Authenticate()

In [ ]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: OUTPUT_BANDS[0]}) + "'"

# Put the EEified model next to the trained model directory.
EEIFIED_DIR = 'gs://' + BUCKET + '/eeified'

# You need to set the project before using the model prepare command.
# if you get config not found; rerun the ee.Authenticate() (above cell)
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

Successfully saved project id
Success: model at 'gs://wa-tf-training/eeified' is ready to be hosted in AI Platform.


### Perform inference using the trained model in Earth Engine

Before it's possible to get predictions from the trained and EEified model, it needs to be deployed on AI Platform.  The first step is to create the model.  The second step is to create a version.  See [this guide](https://cloud.google.com/ml-engine/docs/tensorflow/deploying-models) for details.  Note that models and versions can be monitored from the [AI Platform models page](http://console.cloud.google.com/ai-platform/models) of the Cloud Console.  

In [ ]:
MODEL_NAME = 'wa_tf_training_model'
VERSION_NAME = 'v' + str(int(time.time()))
print('Creating version: ' + VERSION_NAME)

!gcloud ai-platform models create {MODEL_NAME} --project {PROJECT}
!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --runtime-version=2.8 \
  --framework "TENSORFLOW" \
  --python-version=3.7

Creating version: v1692129518
Please specify a region:
(For the global endpoint the region needs to be specified as 'global'.)
 [1] global
 [2] asia-east1
 [3] asia-northeast1
 [4] asia-southeast1
 [5] australia-southeast1
 [6] europe-west1
 [7] europe-west2
 [8] europe-west3
 [9] europe-west4
 [10] northamerica-northeast1
 [11] us-central1
 [12] us-east1
 [13] us-east4
 [14] us-west1
 [15] cancel
Please enter your numeric choice:  11

To make this the default region, run `gcloud config set ai_platform/region us-central1`.

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ai platform model [projects/servir-ee/models/wa_tf_training_model].
Please specify a region:
(For the global endpoint the region needs to be specified as 'global'.)
 [1] global
 [2] asia-east1
 [3] asia-northeast1
 [4] asia-southeast1
 [5] australia-southeast1
 [6] europe-west1
 [7] europe-west2
 [8] europe-west3
 [9] europe-west4
 [10] northamerica-northeast1
 [11] us-central1
 [12] us-east1
 [13] us-e

There is now a trained model, prepared for serving to Earth Engine, hosted and versioned on AI Platform.  We can now connect Earth Engine directly to the trained model for inference.  You do that with the `ee.Model.fromAiPlatformPredictor` command.

#### `ee.Model.fromAiPlatformPredictor`
For this command to work, we need to know a lot about the model.  To connect to the model, you need to know the name and version.

##### Inputs
You need to be able to recreate the imagery on which it was trained in order to perform inference.  Specifically, you need to create an array-valued input from the scaled data and use that for input.  (Recall that the new input node is named `array`, which is convenient because the array image has one band, named `array` by default.)  The inputs will be provided as 16x16 patches (`inputTileSize`), at 30-meter resolution (`proj`), but 8 pixels will be thrown out (`inputOverlapSize`) to minimize boundary effects.

##### Outputs
The output (which you also need to know), is a single float band named `mining`.

In [ ]:
image.select(INPUT_BANDS).bandNames().getInfo()

['VV',
 'VH',
 's1_ratio',
 's1_ndratio',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12']

In [ ]:
# Load the trained model and use it for prediction.
model = ee.Model.fromAiPlatformPredictor(
    projectName = PROJECT,
    modelName = MODEL_NAME,
    version = VERSION_NAME,
    inputTileSize = [16, 16],
    inputOverlapSize = [8, 8],
    proj = ee.Projection('EPSG:4326').atScale(SCALE),
    fixInputProj = True,
    outputBands = {OUTPUT_BANDS[0]: {
        'type': ee.PixelType.float()
      }
    }
)
predictions = model.predictImage(image.select(INPUT_BANDS).toArray())


vis_params = {
  'min': 0,
  'max': 0.4,
  'bands': ['B4', 'B3', 'B2'],
}


map = folium.Map(location=[COORDS[1], COORDS[0]], zoom_start=8)

folium.TileLayer(
    tiles=image.clip(EXPORT_REGION).getMapId(vis_params)["tile_fetcher"].url_format,
    attr="Google Earth Engine",
    overlay=True,
    name="Image",
  ).add_to(map)

folium.TileLayer(
    tiles=predictions.getMapId({'min': 0, 'max': 1})["tile_fetcher"].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='predictions',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

## 3. Using Vertex AI Platform (coming soon ...)

# Next steps . . .